# Lab 3 - Ray and Dask

## Creating an inverted index and Word Counting

Please review Lab 2 before proceeding. Part of this lab is creating an inverted index, but using Ray instead of Parallel. We'll then move onto the more complicated word counting example.

In [2]:
import ray
ray.init(ignore_reinit_error=True)

2023-11-15 20:42:23,160	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-15 20:42:24,672	WARNING utils.py:581 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2023-11-15 20:42:25,834	INFO worker.py:1673 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.8.0


In [3]:
%load_ext autoreload
%autoreload 2


# Put all your solutions into Lab1_helper.py as this script which is autograded
import Lab3_helper
    
import os
from pathlib import Path
home = str(Path.home())

import pandas as pd

## Inverted Index

**Exercise 1:**

You have already written most of what you need to use Ray to construct distributed inverted indices. Here I want you to modify Lab3_helper.py to use Ray and return the final inverted index. I'm supplying the code that divides your books into three sets.

In [4]:
group1 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group1")
group2 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group2")
group3 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group3")

In [5]:
index = Lab3_helper.merge([group1,group2,group3])

In [6]:
index['Education']

{'/home/myuser/csc-369-student/data/gutenberg/group2/45-0.txt': [420655],
 '/home/myuser/csc-369-student/data/gutenberg/group2/408-0.txt': [141091,
  142312,
  142732],
 '/home/myuser/csc-369-student/data/gutenberg/group2/3600-0.txt': [438645,
  549522,
  3030373,
  3030389],
 '/home/myuser/csc-369-student/data/gutenberg/group3/766-0.txt': [351347],
 '/home/myuser/csc-369-student/data/gutenberg/group3/2814-0.txt': [4070],
 '/home/myuser/csc-369-student/data/gutenberg/group3/1399-0.txt': [1285944]}

In [7]:
index['Education']

{'/home/myuser/csc-369-student/data/gutenberg/group2/45-0.txt': [420655],
 '/home/myuser/csc-369-student/data/gutenberg/group2/408-0.txt': [141091,
  142312,
  142732],
 '/home/myuser/csc-369-student/data/gutenberg/group2/3600-0.txt': [438645,
  549522,
  3030373,
  3030389],
 '/home/myuser/csc-369-student/data/gutenberg/group3/766-0.txt': [351347],
 '/home/myuser/csc-369-student/data/gutenberg/group3/2814-0.txt': [4070],
 '/home/myuser/csc-369-student/data/gutenberg/group3/1399-0.txt': [1285944]}

In [8]:
# clean up memory to help us all co-exist on the same machine
index = None
import gc
gc.collect()

0

## Word Counting
Now consider a different problem of common interest. Suppose we have a large corpus (fancy word common in natural language processing) and we want to calculate the number of times a word appears. We could try to hack our inverted index, but let's insert the requirement that this must be a clean implementation. In other words, I'll be manually reviewing your design and making you redo the assignment if it isn't "clean". 

**Exercise 2:**

Write a function that counts the words in a book. Output format shown below. You do not have to worry about punctuation and capitalization. In other words, please stick to simple f.readlines() and line.split(" "). Do not strip anything out.

In [9]:
counts = Lab3_helper.count_words(group1[0])

In [10]:
counts

{'\ufeffThe': 1,
 'Project': 66,
 'Gutenberg': 20,
 'EBook': 2,
 'of': 4047,
 'Jane': 52,
 'Eyre,': 35,
 'by': 586,
 'Charlotte': 4,
 'Brontë\n': 4,
 '\n': 4377,
 'This': 74,
 'eBook': 4,
 'is': 1169,
 'for': 1131,
 'the': 6752,
 'use': 27,
 'anyone': 3,
 'anywhere': 5,
 'in': 2454,
 'United': 12,
 'States': 7,
 'and': 5696,
 'most\n': 13,
 'other': 114,
 'parts': 5,
 'world': 42,
 'at': 1044,
 'no': 483,
 'cost': 5,
 'with': 1297,
 'almost': 87,
 'restrictions\n': 1,
 'whatsoever.': 2,
 '': 141,
 'You': 146,
 'may': 132,
 'copy': 10,
 'it,': 130,
 'give': 107,
 'it': 1431,
 'away': 80,
 'or': 617,
 're-use': 2,
 'under': 133,
 'terms': 25,
 'of\n': 363,
 'License': 8,
 'included': 3,
 'this': 483,
 'online': 4,
 'at\n': 73,
 'www.gutenberg.org.': 2,
 'If': 68,
 'you': 2056,
 'are': 522,
 'not': 1298,
 'located': 7,
 'States,': 3,
 "you'll": 2,
 'have\n': 68,
 'to': 4687,
 'check': 12,
 'laws': 10,
 'country': 14,
 'where': 164,
 'before': 221,
 'using': 7,
 'ebook.\n': 2,
 'Title:': 1

**Exercise 3**

Now let's distribute this using Ray. Please implement a function that parallelizes the word counting and subsequent merges.

In [11]:
merged_counts = Lab3_helper.merge_count_words([group1,group2,group3])

In [12]:
merged_counts

{'\ufeffThe': 44,
 'Project': 4855,
 'Gutenberg': 1405,
 'EBook': 88,
 'of': 234481,
 'Jane': 526,
 'Eyre,': 35,
 'by': 35942,
 'Charlotte': 59,
 'Brontë\n': 8,
 '\n': 203089,
 'This': 5111,
 'eBook': 344,
 'is': 55261,
 'for': 58736,
 'the': 416040,
 'use': 2396,
 'anyone': 844,
 'anywhere': 302,
 'in': 131087,
 'United': 851,
 'States': 490,
 'and': 267326,
 'most\n': 919,
 'other': 9606,
 'parts': 546,
 'world': 2195,
 'at': 49078,
 'no': 19631,
 'cost': 473,
 'with': 70689,
 'almost': 3221,
 'restrictions\n': 36,
 'whatsoever.': 152,
 '': 277065,
 'You': 5326,
 'may': 8645,
 'copy': 667,
 'it,': 8354,
 'give': 5099,
 'it': 63715,
 'away': 4528,
 'or': 29702,
 're-use': 142,
 'under': 5130,
 'terms': 1663,
 'of\n': 18919,
 'License': 592,
 'included': 169,
 'this': 29698,
 'online': 296,
 'at\n': 3545,
 'www.gutenberg.org.': 83,
 'If': 4463,
 'you': 53664,
 'are': 24032,
 'not': 53558,
 'located': 435,
 'States,': 223,
 "you'll": 121,
 'have\n': 3250,
 'to': 226624,
 'check': 346,
 

In [13]:
merged_counts['things']

3499

In [14]:
# Don't forget to push!

(raylet) [2023-11-15 20:47:25,790 E 202240 202240] (raylet) node_manager.cc:3035: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 683a04c2a68bd2a90d0b8100cdd031572cf7535889495aa49518cf71, IP: 172.17.0.52) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.17.0.52`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
